In [27]:
from path_arquivos import *
import pandas as pd
import numpy as np
import glob
import os

In [28]:
def directory(files, argumento):
    directory = glob.glob(os.path.join(files, argumento))
    lista = []
    for arquivo in directory:
        x = pd.read_csv(arquivo, header= None)
        lista.append(x)
    df_temp = pd.concat(lista, axis= 0, ignore_index= True)
    return df_temp

df_sug = directory(pasta.p_82, 'DEP*.txt')
df_mov = directory(pasta.p_82, 'MOV*.txt')
df_prod = pd.read_excel(ar_xlsx.ar_96, usecols=['CODPROD','DESCRICAO', 'QTUNITCX', 'QTTOTPAL', 'OBS2','RUA', 'PREDIO', 'APTO'])
df_acesso = pd.read_excel(ar_xlsx.ar_60, usecols= ['CODPROD','QTOS', 'QT'])
df_estoque = pd.read_excel(ar_xls.ar_86, usecols=['Código', 'Estoque', 'Custo ult. ent.'])


c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [41]:

df_sug.columns = col_name.c82
df_mov.columns = col_name.cMovi

df_prod = df_prod.loc[df_prod['RUA'].between(1,39)]
df_prod['OBS2'] = df_prod['OBS2'].fillna("Ativos")
df_prod["PRODUTO"] = df_prod['CODPROD'].astype(str)

df_estoque = df_estoque.fillna(0)

x = df_acesso.groupby('CODPROD').agg(
ACESSO = ("QTOS", "sum")
).reset_index()

df = df_sug.merge(df_prod, left_on= 'COD', right_on= 'CODPROD', how= 'left')
df = df.merge(df_estoque, left_on='COD', right_on='Código', how='left')
df = df.merge(x, left_on='COD', right_on='CODPROD', how= 'left')
df = df.merge(df_mov, left_on='COD', right_on='COD', how="left")

concat = df['RUA_x'].astype(str) + "-" + df['PREDIO_x'].astype(str)

df[['COM FATOR', 'QTTOTPAL', 'CAP']] = df[['COM FATOR', 'QTTOTPAL','CAP']].fillna(0)
df['PL_SUG'] = (df['COM FATOR'].astype(int) / df['QTTOTPAL'].astype(int)).round(2)
df['PL_CAP'] = (df['CAP'].astype(int) / df['QTTOTPAL'].astype(int)).round(2)
df['Custo ult. ent.'] = df['Custo ult. ent.'].astype(float).round(2)
df['CUSTO_TT'] = (df['Estoque'].astype(float) * df['Custo ult. ent.']).round(2)
df['CONTAGEM'] = concat.map(concat.value_counts())
df['STATUS_PROD'] = np.where(df['CONTAGEM'] > 3, "DIV", np.where(df['CONTAGEM'] > 2,"VAL", "INT"))


In [42]:
drop_col = ['UNID.','DESCRIÇÃO', '%_x',  'CODPROD_x','RUA_y', 'PREDIO_y', 'APTO_y','Código', 'CODPROD_y', 'ACESSO','DESCRICAO','EMBALAGEM_y', 'UNID','%_y', '%_ACUM','NIVEL']
df_final = df.drop(columns=drop_col)

dic_rename = {
    "EMBALAGEM_x" : "EMB"
    ,"RUA_x" : "RUA"
    ,"PREDIO_x" : "PREDIO"
    ,"APTO_x" : "APTO"
    ,"Custo ult. ent." : "CUSTO_ULT"
}
df_final = df_final.rename(columns=dic_rename)

nova_ordem = ['COD', 'DESC','EMB', 'OBS2', 'QTUNITCX','QTTOTPAL', 'RUA', 'PREDIO', 'APTO', 'TIPO', 'MÊS 1', 'MÊS 2', 'MÊS 3', 'CAP', '1 DIA', 'COM FATOR', 'VARIAÇÃO','Estoque', 'CUSTO_ULT','MOVI', 'CLASSE', 'PL_SUG', 'PL_CAP', 'CONTAGEM', 'STATUS_PROD', 'CUSTO_TT','PRODUTO']
df_final = df_final[nova_ordem]

In [43]:
display(df_final.columns)
display(df_final.dtypes)
display(df_final.head())

Index(['COD', 'DESC', 'EMB', 'OBS2', 'QTUNITCX', 'QTTOTPAL', 'RUA', 'PREDIO',
       'APTO', 'TIPO', 'MÊS 1', 'MÊS 2', 'MÊS 3', 'CAP', '1 DIA', 'COM FATOR',
       'VARIAÇÃO', 'Estoque', 'CUSTO_ULT', 'MOVI', 'CLASSE', 'PL_SUG',
       'PL_CAP', 'CONTAGEM', 'STATUS_PROD', 'CUSTO_TT', 'PRODUTO'],
      dtype='object')

COD              int64
DESC            object
EMB             object
OBS2            object
QTUNITCX         int64
QTTOTPAL         int64
RUA              int64
PREDIO           int64
APTO             int64
TIPO            object
MÊS 1            int64
MÊS 2            int64
MÊS 3            int64
CAP              int64
1 DIA            int64
COM FATOR        int64
VARIAÇÃO         int64
Estoque        float64
CUSTO_ULT      float64
MOVI           float64
CLASSE          object
PL_SUG         float64
PL_CAP         float64
CONTAGEM         int64
STATUS_PROD     object
CUSTO_TT       float64
PRODUTO         object
dtype: object

,COD,DESC,EMB,OBS2,QTUNITCX,QTTOTPAL,RUA,PREDIO,APTO,TIPO,...,Estoque,CUSTO_ULT,MOVI,CLASSE,PL_SUG,PL_CAP,CONTAGEM,STATUS_PROD,CUSTO_TT,PRODUTO
0,473137,BALAS REGALIZ FINI 80GR TUBES SORTIDOS,CX/0012/UN,Ativos,1,510,13,34,102,"INTEIRO(1,90)",...,1525.0,42.11,1356.0,B,0.12,1.07,2,INT,64217.75,473137
1,473135,BALAS GELAT FINI 80GR AROS DE MORANGO,CX/0012/UN,Ativos,1,364,13,30,101,"INTEIRO(1,90)",...,2354.0,42.11,3810.0,A,0.43,1.07,2,INT,99126.94,473135
2,473125,BALAS GELAT FINI 80GR MINHOCAS CITRICAS,CX/0012/UN,Ativos,1,364,13,32,101,"INTEIRO(1,90)",...,2932.0,42.11,3724.0,A,0.43,1.07,2,INT,123466.52,473125
3,473149,MARSHMALLOW FINI 80GR CAMPING,CX/0012/UN,Ativos,1,126,13,36,102,"INTEIRO(1,90)",...,1140.0,42.11,1226.0,B,0.40,1.11,2,INT,48005.40,473149
4,473129,BALAS GELAT FINI 80GR BEIJOS DE MORANGO,CX/0012/UN,Ativos,1,364,13,22,101,"INTEIRO(1,90)",...,4895.0,42.11,6117.0,A,0.75,1.07,2,INT,206128.45,473129
